In [7]:
import pandas as pd
import time
import nba_api.stats.endpoints as nbaapi
from nba_api.stats.endpoints import *
import rdshelp
import json

# Define parameters at the top - papermill will inject these values
param1 = "BoxScoreAdvancedV3"  # Default value
param2 = "0"  # Default value
testgame = '0022201200'

conn = rdshelp.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')

def games_createandinsert(func,index):
    print(f"Processing data for: {func,index}")
    func = getattr(nbaapi,func)
    index = int(index)
    param = testgame
    dfname = list(func.expected_data.keys())[index]

    # need to replace with the endpoint class
    namepre = func.__name__.lower()
    name = namepre+"_"+dfname.lower()

    df = func(param).get_data_frames()[index]

    ### Creates Table if none exists
    rdshelp.create_table(conn,name,df)

    gameslist = rdshelp.game_difference(conn,name)

    for i in gameslist:
        try:
            tdf = func(i).get_data_frames()[index]
            rdshelp.insert_dataframe_to_rds(conn,tdf,name)
            time.sleep(1)
        except Exception as e:
            fdf = pd.DataFrame({"gameid":[i]})
            rdshelp.insert_dataframe_to_rds(conn,fdf,name)
            print("An error occurred:", e)

games_createandinsert(param1,param2)

Connected to RDS PostgreSQL database
Processing data for: ('BoxScoreAdvancedV3', '0')
Master table boxscoreadvancedv3_playerstats created successfully.


AttributeError: module 'rdshelp' has no attribute 'game_difference'

In [4]:

print(rdshelp.__file__)

c:\Users\ajwin\Projects\NBA\thebigone\.venv\Lib\site-packages\rdshelp.py
